In [1]:
library(tidyverse)

#install.packages("tidyr")
library(tidyr)

#install.packages('scales')
library(scales)

#install.packages("slider")
library(slider)


── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──
✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘scales’

The following object is masked from ‘package:purrr’:

    discard

The following object is masked from ‘package:readr’:

    col_factor



In [2]:
filename = snakemake@input[["all_samples"]]

In [3]:
small_test <- read_csv(file = filename, col_names = T, comment = "#", skip = 0, n_max = 10)
head(small_test)

sample,bin,X30,X31,X32,X33,X34,X35,X36,X37,...,X691,X692,X693,X694,X695,X696,X697,X698,X699,X700
PGDX10344P1,chr10_400,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGDX10344P1,chr10_40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGDX10344P1,chr10_4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGDX10344P1,chr10_401,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGDX10344P1,chr10_402,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGDX10344P1,chr10_403,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
cols_smoothed = as.data.frame(t(c(colnames(small_test))))
cols_smoothed

In [5]:
write.table(NULL, file=snakemake@output[["csv_output"]], 
             append = F, 
             sep=',', 
             row.names=F, 
             col.names=F)

write.table(cols_smoothed, snakemake@output[["csv_output"]], append = F, sep=',', col.names=F, row.names=F)

In [ ]:
f2 <- function(df1, pos) {
    df1[3:673] <- lapply(df1[3:673], as.numeric)
    df <- df1 %>% select(-c("sample", "bin")) %>% as.matrix()
    output <- matrix(ncol = 671, nrow = 0)

    for (row in 1:nrow(df)){
        row_cur <- df[row, ]
        row_cur <- as.vector(t(row_cur))
        roll_m <- slide_dbl(row_cur, mean, .before = 5, .after = 5)
        output <- rbind(output, roll_m)
    }

    output <- data.frame(df1$sample, df1$bin, output)

    write.table(output, snakemake@output[["csv_output"]], append = TRUE, sep=',', col.names=F, row.names=F)
    }


df <- read_delim_chunked(file = filename, 
                         callback = DataFrameCallback$new(f2), 
                         chunk_size = 10000, col_names = T,
                         delim = ",", 
                         progress = show_progress())